In [4]:
from bs4 import BeautifulSoup
import requests
from requests import session, get
from urllib.parse import urljoin
import shutil
import re
import pathlib
from typing import Dict, List


Wir wollen alle `*.pdf` Dateien [dieser Seite](https://www.wim.uni-mannheim.de/schlather/teaching/aktuelle-semester/) finden und automatisiert herunterladen. Dazu bauen
wir uns eine Klasse `Scraper`.

1. Verschaffe dir mittels Chromes Entwicklertools einen Überblick über die Loginform. Welche Daten werden alles benötigt und abgesendet?
2. Implementiere die Methode `_materialien_login() -> None`, welche sich mittels einer `requests.session` auf der Seite einloggt und alle Cookies speichert. Die Session soll im Klassenattribut `_session` gespeichert werden. Achtung: Es gibt zwei HTML-Forms auf der Seite.
3. Implementiere die Methode `_scrape_pdfs() -> None`, welche für jede Vorlesung auf der Materialienseite die Links zu den pdf-Dateien findet. Die Methode speichert alle Links pro Vorlesung in `self.daten`. Hinweis: Verwende `.replace("\xad", "")` um HTML entities aus dem Vorlesungsnamen zu entfernen.
4. Teste den Code und lade alle Dateien herunter :)

In [5]:
class Scraper:
    def __init__(self):
        self._session: requests.session = None
        self.daten: Dict[str, List[str]] = {}
    
    def __del__(self):
        if self._session is not None:
            self._session.close()

    def _materialien_login(self) -> None:
        login_url = ("https://www.wim.uni-mannheim.de"
                     "/schlather/teaching/aktuelle-semester/")
        soup = BeautifulSoup(get(login_url).text, "lxml")
        # TODO:

    def _scrape_pdfs(self):
        base_url = ("https://www.wim.uni-mannheim.de"
                    "/schlather/teaching/aktuelle-semester/materialien/")
        # TODO:

    def _download_file(self, url: str, pfad: pathlib.Path, filename: str) -> None:
        req = self._session.get(url, stream=True)
        if req.status_code == 200:
            with open(pfad / filename, "wb") as f:
                shutil.copyfileobj(req.raw, f)

    def download_all_files(self, verbose=False):
        self._materialien_login()
        self._scrape_pdfs()

        for vl_name, links in self.daten.items():
            # Erstelle Ordner für VL, falls dieser noch nicht existiert
            p = pathlib.Path().cwd() / vl_name.replace(":", "")
            if not p.exists():
                p.mkdir()
            for link in links:
                dateiname = link.split("/")[-1]
                if verbose:
                    print(f"Lade [{vl_name}]: {dateiname}...")
                self._download_file(link, p, dateiname)
